In [1]:
import os
import pyproj
import rasterio
import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm
import geopandas as gpd
from rasterio.features import rasterize
from rasterio.transform import from_origin, from_bounds

print('Modules imported.')

Modules imported.


In [2]:
relevant_columns = [
    'occurrenceID', 
    'verbatimScientificName', 
    'decimalLatitude', 
    'decimalLongitude', 
    'coordinateUncertaintyInMeters', 
    'eventDate'
]

gbif_data = pl.read_csv(
    r'D:\Projects\Project-1\GBIF-Data\gbif-data-with-expanded-dates.csv',
    separator=',',
    columns=relevant_columns
)

species_index = (
    gbif_data
    .select('verbatimScientificName')
    .unique()
    .sort('verbatimScientificName')
    .with_row_index('species_index')
)

species_index.head()

species_index,verbatimScientificName
u32,str
0,"""Abies"""
1,"""Abies alba"""
2,"""Abies alba × concolor"""
3,"""Abies balsamea"""
4,"""Abies cephalonica"""


In [3]:
gbif_data = gbif_data.join(
    species_index, 
    on="verbatimScientificName", 
    how="left"
)

gbif_data.head()

occurrenceID,verbatimScientificName,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,eventDate,species_index
str,str,f64,f64,f64,str,u32
"""urn:lsid:artportalen.se:sighti…","""Cota tinctoria""",59.75263,17.0766,261.0,"""2019-07""",1885
"""urn:lsid:artportalen.se:sighti…","""Rosa vosagiaca""",56.6378,16.54385,50.0,"""2019-07""",6718
"""urn:lsid:artportalen.se:sighti…","""Vaccinium myrtillus""",58.56746,15.71282,50.0,"""2019-07""",8660
"""urn:lsid:artportalen.se:sighti…","""Campanula rapunculoides""",60.29459,13.87982,150.0,"""2019-07""",1109
"""urn:lsid:artportalen.se:sighti…","""Trifolium medium""",58.56221,14.93347,100.0,"""2019-07""",8522


In [4]:
grid_resolution = 5000 # in meters
ROI_bounds = [
    -224767,            # x_min
    6083545,            # y_min
    1413055,            # x_max
    7721367,            # y_max
]

x_min, y_min, x_max, y_max = ROI_bounds
# width = int((x_max - x_min) / grid_resolution)
# height = int((y_max - y_min) / grid_resolution)
width = 328
height = 329
transform = from_bounds(x_min, y_min, x_max, y_max, width, height)

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:32633')
transformer = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True)

print(f'Raster size: {width}x{height}')

output_folder = '../occurrence-rasters'
os.makedirs(output_folder, exist_ok=True)

Raster size: 328x329


In [5]:
event_groups = gbif_data.group_by('eventDate')
species_indices = gbif_data['species_index'].unique().to_list()

for event_date, group in tqdm(event_groups, desc="Processing event dates"):
    
    event_date = event_date[0]
    # print(f'Processing data for: {event_date}')
    raster = np.zeros((len(species_indices), width, height), dtype=np.uint8)

    x_coords, y_coords = transformer.transform(group['decimalLongitude'].to_numpy(), group['decimalLatitude'].to_numpy())

    col_idx = ((x_coords - x_min) / grid_resolution).astype(int)
    row_idx = ((y_max - y_coords) / grid_resolution).astype(int)

    for idx, row in tqdm(enumerate(group.iter_rows(named=True)), total=len(group), desc=f"Processing rows for {event_date}", leave=False):

        species_idx = row['species_index']

        c = col_idx[idx]
        r = row_idx[idx]

        if 0 <= r < height and 0 <= c < width:
            raster[species_idx, r, c] = 1

    output_path = os.path.join(output_folder, f'occurrence_{event_date}.tif')
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=len(species_indices),
        dtype=raster.dtype,
        crs='EPSG:32633',
        transform=transform,
        compress='LZW'
    ) as dst:
        for band_idx in range(len(species_indices)):
            dst.write(raster[band_idx, :, :], band_idx + 1)


Processing event dates: 0it [00:00, ?it/s]

Processing rows for 1990-03:   0%|          | 0/51653 [00:00<?, ?it/s]

Processing rows for 2015-01:   0%|          | 0/8026 [00:00<?, ?it/s]

Processing rows for 1980-04:   0%|          | 0/33117 [00:00<?, ?it/s]

Processing rows for 2014-08:   0%|          | 0/27010 [00:00<?, ?it/s]

...

Processing rows for 1986-05:   0%|          | 0/83049 [00:00<?, ?it/s]

Processing rows for 2000-12:   0%|          | 0/44780 [00:00<?, ?it/s]

Processing rows for 1987-11:   0%|          | 0/40855 [00:00<?, ?it/s]

Processing rows for 2015-11:   0%|          | 0/10633 [00:00<?, ?it/s]

Processing rows for 2009-02:   0%|          | 0/5731 [00:00<?, ?it/s]